# Matrix Porridge
by Kyle Archie, M.Eng

This work builds upon the first two notebooks in this repository. Please read through that one for background and explanation of the data / process.

## Notebook 4: Global Optimizer and Hard Constraints

I've done a few things since notebook 3:
1.  that instead of using the clustering method to filter out similar items, I manually went through and picked the most healthy sounding of the options... that is to say, those that did not have added fat or salt, and those items that were marked as either 'lean only eaten' for meat options and reduced sodium. If I were starting this from scratch, I might not have made those choices, but a common thread in the solutions I've seen so far is that they struggle with both the sodium and saturated fat requirements. 
1.  Also, I've done a fair amount of reading on the dietary requirements of salt as I'm becoming increasingly suspicious that it might not be actually possible to meet the current daily requirements while also meeting the other core requirements. I found a number of studies that suggest that the research on salt is far from conclusive. Scientific American had an [article](https://www.scientificamerican.com/article/its-time-to-end-the-war-on-salt/#:~:text=Not%20necessarily.,normal%20or%20high%20blood%20pressure.) on this in 2011 where they discussed a meta analysis of 7 studies published in the Journal of Hypertension, but there are many others, albeit from potentially less reliable sources. The general consensus from government agencies around the world does appear to be consistent with the requirements we're using, but my findings so far along with this Scientific American article are leading me to want to investigate this a bit more.
1.  To give the salt requirement one last show, I added it as a hard constraint. After some iterations with this notebook, I also added hard constraints for the total weight of the solution, explained in more detail later.
1.  I modified the objective function to effectively round to zero and ingredients below a certain weight threshold to trim the solution down to something with a reasonable number of ingredients. No more word cloud needed to show the output.
1.  I used a differential evolution algorithm for the global optimizer.

In [1]:
import pandas as pd

ingredients = pd.read_excel("FoodData_Export_culled.xlsx", "filtered")
ingredients['Sodium, Na']=ingredients['Sodium, Na']/1000 #values are in mg... convert to g
ingredients.set_index('description',inplace=True)
ingredients['Total Fat']=ingredients[['Fatty acids, total saturated','Fatty acids, total monounsaturated','Fatty acids, total polyunsaturated']].sum(axis=1)
ingredients=ingredients[['Energy','Protein','Carbohydrate, by difference','Fiber, total dietary','Sugars, total including NLEA','Total Fat','Fatty acids, total saturated','Fatty acids, total polyunsaturated','Sodium, Na']].dropna()

Reference used: https://towardsdatascience.com/k-means-vs-dbscan-clustering-49f8e627de27

In [2]:
import numpy as np

In [3]:
inputs=ingredients.T
# inputs.columns=ingredients['Shrt_Desc']
# inputs.dropna(inplace=True,axis=1) #some of these ingredients have null values. Remove those. 
inputs.fillna(0,inplace=True)
inputs.head(10)

description,"Apple juice, 100%","Applesauce, unsweetened","Apple, raw","Apple, baked","Pork bacon, NS as to fresh, smoked or cured, reduced sodium, cooked","Beef, bacon, reduced sodium, cooked","Pork bacon, smoked or cured, cooked","Bacon or side pork, fresh, cooked","Pork bacon, smoked or cured, reduced sodium, cooked","Salt pork, cooked",...,"Tomato juice, 100%, low sodium","Tomato and vegetable juice, 100%, low sodium",Celery juice,"Yogurt, Greek, whole milk, plain","Yogurt, Greek, low fat milk, plain","Yogurt, Greek, nonfat milk, plain","Yogurt, whole milk, plain","Yogurt, low fat milk, plain","Yogurt, nonfat milk, plain","Yogurt, coconut milk"
Energy,46.000,42.000,52.000,112.000,541.000,541.000,468.000,468.000,541.000,750.000,...,22.000,22.000,14.000,97.000,73.000,61.000,61.000,63.000,56.000,64.000
Protein,0.100,0.170,0.260,0.320,37.040,37.040,33.920,33.920,37.040,2.680,...,0.600,0.600,0.690,9.000,9.950,10.300,3.470,5.250,5.730,0.310
"Carbohydrate, by difference",11.300,11.270,13.810,22.700,1.430,1.430,1.700,1.700,1.430,0.000,...,4.590,4.590,2.970,3.980,3.940,3.640,4.660,7.040,7.680,7.950
"Fiber, total dietary",0.200,1.100,2.400,2.500,0.000,0.000,0.000,0.000,0.000,0.000,...,0.800,0.800,1.600,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"Sugars, total including NLEA",9.620,9.390,10.390,18.990,0.000,0.000,0.000,0.000,0.000,0.000,...,3.280,3.280,1.340,4.000,3.560,3.270,4.660,7.040,7.680,7.400
Total Fat,0.067,0.024,0.086,2.806,36.807,36.807,33.606,33.606,36.807,78.723,...,0.072,0.072,0.153,5.000,1.792,0.162,3.081,1.470,0.170,3.506
"Fatty acids, total saturated",0.022,0.008,0.028,1.812,13.739,13.739,11.964,11.964,13.739,31.339,...,0.014,0.014,0.042,2.395,1.230,0.108,2.096,1.000,0.116,3.422
"Fatty acids, total polyunsaturated",0.039,0.014,0.051,0.160,4.548,4.548,6.112,6.112,4.548,10.744,...,0.042,0.042,0.079,0.469,0.076,0.011,0.092,0.044,0.005,0.016
"Sodium, Na",0.004,0.002,0.001,0.004,1.030,1.030,1.684,1.684,1.030,2.039,...,0.058,0.058,0.080,0.035,0.034,0.036,0.046,0.070,0.077,0.021


In [4]:
print(ingredients.columns)

Index(['Energy', 'Protein', 'Carbohydrate, by difference',
       'Fiber, total dietary', 'Sugars, total including NLEA', 'Total Fat',
       'Fatty acids, total saturated', 'Fatty acids, total polyunsaturated',
       'Sodium, Na'],
      dtype='object')


In [5]:
requirements=pd.read_excel("Matrix Porridge (filtered).xlsx", "requirements (2021)")
requirements['min (g)']/=3
requirements['max (g)']/=3

print(requirements)

                                          Unnamed: 0    min (g)     max (g)
0                                                Fat  14.814815   25.925926
1   n-6 polyunsaturated fatty acidsa (linoleic acid)   3.703704    7.407407
2  n-3 polyunsaturated fatty acidsa (α-linolenic ...   0.444444    0.888889
3                                       Carbohydrate  75.000000  108.333333
4                                            Protein  16.666667   58.333333
5                                             Sodium   0.500000    0.766667


Note: our data does not split the two kinds of polyunsaturated fat, so we'll have to sum these up. There's another dataset I've been looking at which might work better for this, but we'll get to that later.

In [6]:
fat_min=requirements.iloc[0,1]
fat_max=requirements.iloc[0,2]
fat_half_range=(fat_max-fat_min)/2 #calculate this once so we don't need to do it repeatedly in our function later
fat_opt=(fat_min+fat_max)/2

pufat_min=requirements.iloc[1,1]+requirements.iloc[2,2]
pufat_max=requirements.iloc[1,2]+requirements.iloc[2,2]
pufat_half_range=(pufat_max-pufat_min)/2
pufat_opt=(pufat_min+pufat_max)/2

carb_min=requirements.iloc[3,1]
carb_max=requirements.iloc[3,2]
carb_half_range=(carb_max-carb_min)/2
carb_opt=(carb_min+carb_max)/2

protein_min=requirements.iloc[4,1]
protein_max=requirements.iloc[4,2]
protein_half_range=(protein_max-protein_min)/2
protein_opt=(protein_min+protein_max)/2

sodium_min=requirements.iloc[5,1]
sodium_max=requirements.iloc[5,2]
sodium_half_range=(sodium_max-sodium_min)/2
sodium_opt=(sodium_min+sodium_max)/2

## The Approach
This is clearly an optimization problem. However, it is a bit more complicated than what you'd typically use Linear Programming to solve. We could potentially frame it that way... with an A matrix 2330 columns wide. But our objective function here isn't written easily as a function of pure X (our ingredients vector), no matter what we decide to optimize.

There are, however, many modern machine learning approaches that can help us here. The trick is to use a solver / algorithm that works with a custom function instead of a vector. This way, we can to optimize for a custom value function. SciPy's Linear Programming functionality requires that our objective be a vector, but that doesn't really work here. But there are plenty of alternatives, so we'll try a few of those. 

First, however, we need to define what it is we seek to optimize. Eventually, we may wish to make this something the user could select from a list of options (which would also alter constraints), to accommodate different nutrition guides, such as Atkins, or a high fiber diet. Or, maybe we seek to maximize quantity of food while still meeting nutrition guidelines. For now, I'm going to take bit of a fuzzy logic approach, with our function outputting a value that's most optimal when all nutrition requirements are exactly in the center of the ranges and where we impose serious (but linear with a slope) penalties if any nutritional requirements fall outisde the acceptable ranges.

In [7]:
import numpy as np
x=np.zeros(len(ingredients))
x[1]=1
A=inputs.values #get A matrix
y=A.dot(x)
print(y)
print(np.count_nonzero(x))

[4.200e+01 1.700e-01 1.127e+01 1.100e+00 9.390e+00 2.400e-02 8.000e-03
 1.400e-02 2.000e-03]
1


# Constraints
In previous attempts, I have treated this as an unconstrained problem, building in some 'soft' constraints to my objective function. That is, I more heavily penalized solutions where certain nutritional values are outside the recommended bounds. However, I've made some serious progress since notebook 3 with finding a more globally optimimal solution. And, I've figured out how to trim the ingredients list down by zeroing out any input values to the objective function below a certain threshold (and subsequently zeroing out anything from the solution in the same way). 

But, I have two problems:
1.  many of my solutions contain a massive amount of food (~5Kg). More than a person could realistically eat. This also got me thinking that, even though I haven't seen it, there could be a niche for this product with people who are interested in losing weight, especially if the final recipe ends up being palatable. So, we should constrain our solution to be bound between a minimum and maximum total weight. More research should probably be done here, but some quick searching led me [here](https://www.stack.com/a/forget-calories-the-weight-of-your-food-is-what-really-matters). This site says most people eat between 3-5 lbs per day. Assuming 3 meals per day, that's 1-1.67lbs or 0.45 to 0.76 Kg per meal. Recall, however that our units here are all values "per 100g" so this means if the sum of our x vector is 1.0, we have 100g. Thus, we n use 4.5 (4.5*100=450g) as our lower bound and 7.6 (7.6*100=760g) as our upperbound.
1.  Salt. I have yet to find a solution that does not exceed the maximum recommended sodium value.

So, now I'm going to add these as true constraints. We'll make a matrix A_c such that lb<=A_c*x<=ub where lb is the lower bound and ub is the upper bound of each constraint.

In [8]:
from scipy.optimize import LinearConstraint

In [9]:
#make a row of ones for the weight constraints:
row1=np.ones(len(ingredients)).T
row2=A[8,:]
A_c=np.stack((row1,row2),axis=1).T
# constr=LinearConstraint(A_c,lb=[4.5],ub=[7.6])
constr=LinearConstraint(A_c,lb=[4.5,sodium_min],ub=[7.6,sodium_max])

# New Objective Function
This version will both filter small values in objective function to trim ingredients as described earlier as well as remove the salt consideration.

In [10]:
threshold=0.01


def diet_function(ingredients_vector):
    max_acceptable_ingredients=50
    ingredients_vector=np.where(ingredients_vector<threshold,0,ingredients_vector)
    ingredient_penalty=max(0,np.count_nonzero(ingredients_vector)-max_acceptable_ingredients)

        
    ingrdient_weight=0.1
    y=A.dot(ingredients_vector)
    calorie_penalty=(666.7-y[0])**2 #weighting calories very highly here
    
    protein_penalty=abs(y[1]-protein_opt)
    if abs(y[1]-protein_opt)>protein_half_range:
        protein_penalty*=protein_penalty
    
    carb_penalty=abs((y[2]+y[3])-carb_opt)
    if carb_penalty>carb_half_range:
        carb_penalty*=carb_penalty    
#     fiber_bonus=y[3]
    sugar_penalty=y[4]**2

        
    fat_penalty=abs(y[5]-fat_opt)
    if fat_penalty>fat_half_range:
        fat_penalty*=fat_penalty
       
    sat_fat_penalty=y[6]**2
    
    pufat_penalty=abs(y[7]-pufat_opt)
    if pufat_penalty>pufat_half_range:
        pufat_penalty*=pufat_penalty
           
    sodium_penalty=abs(y[8]-sodium_opt)
    if sodium_penalty>sodium_half_range:
        sodium_penalty*=sodium_penalty*10 #adjusting for small number         
            
    value=10*ingredient_penalty+calorie_penalty+protein_penalty+carb_penalty+sugar_penalty+fat_penalty+sat_fat_penalty+pufat_penalty+sodium_penalty
    return value 
        

Note: The major change from v3 to v4 is we now add the number of ingredients to the output of the value function. Hopefully, this will serve to reduce the number of total ingredients in the end, though we might need to add a weight term to this to tune it.

In [11]:
from scipy.optimize import minimize,dual_annealing

In [12]:
bounds=tuple([(0,10) for i in range(len(ingredients))])
x0=[5]*len(ingredients)

Note: trying a different initial condition here... .05 instead of 1

### Local vs Global Optimization
SciPy's minimize function is a local optimization algorithm, with many different methods you can choose from to find a local minima based on various methods depending on whether you have bounded inputs or other constraints. 

For those that unfamiliar with local vs global optimization concepts, Mathworks (the makers of Matlab) explains it quite well [here](https://www.mathworks.com/help/gads/what-is-global-optimization.html).

In a nutshell, because of the way we set up our value function with different slopes / contributions to the overall value for each nutritional category depending on whether they are inside our outside our acceptable ranges, we have made this into a nonlinear problem. What that means is that if we start at a random initial position on our value function and use something like gradient descent or Newton's method (or various other approaches) to follow the slope to the local minima, we can't be sure that this is the same as the overall or global minima, which is the true optimal solution. Our starting point and various other hyperparameters (like learning rate) matter. Check out the following graph for a visual explanation:

<img src="https://www.mathworks.com/help/gads/local_vs_global.png">

So the way that we typically go about finding the true optimal solution for these sorts of problems is to use a local optimizer with a multitude of initial starting conditions. Those starting conditions can be purely random, or they can follow some sort of search logic. SciPy offers several options. Generally, I find the dual_annealing offers a good overall performance here. But, I'm going to hold off on running that for now because it's quite slow, and as you'll see later, we have a lot of work left to do before we're ready to go for the final run.

# New Global Optimizer Used
In this iteration, I used SciPy's [Differential Evolution Algorithm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution), which let me use my nonlinear objective function along with bounds on my inputs and linear constraints for sodium and weight. I found this solution to work much better than dual annealing, at least for this problem. For one, it runs quicker, perhaps because it actually seems to make use of multithreading across multiple processors, unlike dual annealing which only uses one. It's also a stochastic method which doesn't require any fanciness with the Jacobian or Hessian matrices or approximation algorithms. 


In [13]:
from scipy.optimize import BFGS,differential_evolution


In [14]:
# %%time
# res=minimize(diet_function,x0,method='trust-constr',jac='3-point',hess=BFGS(),bounds=bounds,constraints=constr,tol=1e-2,options={'maxiter':int(1e9)})

In [15]:
%%time
res=differential_evolution(diet_function,bounds=bounds,constraints=constr)

C:\Users\kylea\anaconda3\envs\jokia\lib\site-packages\scipy\optimize\_differentialevolution.py:805: UserWarning: differential evolution didn't find a solution satisfying the constraints, attempting to polish from the least infeasible solution
  warnings.warn("differential evolution didn't find a"
C:\Users\kylea\anaconda3\envs\jokia\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Wall time: 2h 44min 58s


In [16]:
%%time
# res=dual_annealing(diet_function,x0=x0,bounds=bounds,constraints=constr,maxiter=int(1e5),maxfun=1e12,local_search_options={'method':'trust-constr','jac':'cs','hess':BFGS(),'tol':1e-2,'options':{'maxiter':int(1e3),'minfev':0}})

Wall time: 0 ns


In [17]:
print(res)

           constr: [array([0., 0.])]
 constr_violation: 0.0
              fun: 4315.577835807454
              jac: [array([[1.   , 1.   , 1.   , ..., 1.   , 1.   , 1.   ],
       [0.004, 0.002, 0.001, ..., 0.07 , 0.077, 0.021]]), array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])]
            maxcv: 0.0
          message: 'Maximum number of iterations has been exceeded.'
             nfev: 331936
              nit: 1000
          success: False
                x: array([8.70030273e-10, 6.71194717e-10, 1.29244161e-09, 1.15547578e-09,
       2.03303471e-10, 1.75220527e-10, 3.27011766e-10, 3.22853882e-10,
       4.56182256e-10, 2.82311142e-10, 5.01707649e-10, 1.13221644e-09,
       8.30755100e-10, 9.83624734e-10, 2.85096164e-10, 3.42721755e-10,
       6.19418701e-10, 7.42462452e-10, 9.19642310e-10, 7.

In [18]:
solution=res.x
solution=np.where(solution<threshold,0,solution)


In [19]:

output=pd.Series(A.dot(solution),index=inputs.index)
print(output)

Energy                                652.893754
Protein                                36.535675
Carbohydrate, by difference           100.427392
Fiber, total dietary                    4.432852
Sugars, total including NLEA           62.930677
Total Fat                              13.597673
Fatty acids, total saturated            8.573607
Fatty acids, total polyunsaturated      0.874175
Sodium, Na                              0.766667
dtype: float64


Note: unlike previous iterations (many of which were attempted post notebook 3, so you'll have to take my word for it...), this solution actually meets our salt and weight constraints. Using those hard constraints seems to have worked. However, the saturated fat content is certainly higher than would be ideal. So the question becomes, which is worse; salt or saturated fat?

In [20]:
solution_ds=pd.Series(solution*100,index=ingredients.index,name="grams") #multiply by 100 for grams
solution_ds=solution_ds[solution_ds>0.]
solution_ds.to_csv('matrix_porridge_recipe_v4.csv')
print(solution_ds)
print(sum(solution_ds))

description
Milk, low sodium, whole       387.271273
Ripe plantain, raw            260.756015
Lobster, steamed or boiled    111.972658
Name: grams, dtype: float64
759.9999465689441


Milk and lobster aren't an unheard of combination... makes me think of a bisque. But raw plantain? I don't know how edible that really is. It's almost always eaten cooked. You'd really need to have no sense of taste I think to be able to down this.

In [21]:
A_c.dot(solution)

array([7.59999947, 0.76666655])

## Next Steps

1.  Do more research into both the salt and saturated fat dietary requirements, and consider modifying one or the other to allow more flexibility, or prioritize one over the other.
1.  Revisit the ingredients list and consider culling more items from the list based on edibility and shelf life / food safety. I realize this is supposed to be for people who can't taste, but it may be unrealistic to include raw fish, and wild game meat, as well as certain raw produce, like artichokes and plantains.